# stablediffusion-infinity

https://github.com/lkwq007/stablediffusion-infinity

Outpainting with Stable Diffusion on an infinite canvas

## import libs

In [ ]:
import time

from ipycanvas import Canvas, hold_canvas, MultiCanvas
import ipywidgets as widgets

from utils import *
from canvas import InfCanvas

In [ ]:
import numpy as np
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, StableDiffusionInpaintPipeline

## setup StableDiffusionInpaintPipeline

In [ ]:
text2img = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True).to("cuda")
inpaint=StableDiffusionInpaintPipeline(
            vae=text2img.vae,
            text_encoder=text2img.text_encoder,
            tokenizer=text2img.tokenizer,
            unet=text2img.unet,
            scheduler=text2img.scheduler,
            safety_checker=text2img.safety_checker,
            feature_extractor=text2img.feature_extractor
).to("cuda")

## setup UI control

In [ ]:
strength_slider = widgets.FloatSlider(
    value=0.75,
    min=0,
    max=1.0,
    step=0.01,
    description="Strength:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
    # layout=widgets.Layout(width='100px')
)

step_input = widgets.BoundedIntText(
    value=50,
    min=1,
    max=1000,
    step=1,
    description="Steps:",
    disabled=False,
    layout=widgets.Layout(width="180px"),
)
guidance_input = widgets.FloatText(
    value=7.5,
    step=0.1,
    description="Guidance:",
    disabled=False,
    layout=widgets.Layout(width="180px"),
)
resize_check = widgets.Checkbox(
    value=True,
    description="Resize SD input to 512x512",
    disabled=False,
    indent=False,
    layout=widgets.Layout(width="180px"),
)


## setup InfCanvas

In [ ]:
base = InfCanvas(800, 600, selection_size=256)
base.setup_widgets()
base.setup_mouse()


def run_outpaint(btn):
    with base.output:
        base.output.clear_output()
        base.read_selection_from_buffer()
        img = base.sel_buffer[:, :, 0:3]
        mask = base.sel_buffer[:, :, -1]
        process_size = 512 if resize_check.value else base.selection_size
        if mask.sum() > 0:
            img, mask = functbl[base.fill_button.value](img, mask)
            init_image = Image.fromarray(img)
            mask = 255 - mask
            # mask=skimage.measure.block_reduce(mask,(8,8),np.max)
            # mask=mask.repeat(8, axis=0).repeat(8, axis=1)
            mask_image = Image.fromarray(mask)
            # mask_image=mask_image.filter(ImageFilter.GaussianBlur(radius = 8))
            with autocast("cuda"):
                images = inpaint(
                    prompt=base.text_input.value,
                    init_image=init_image.resize(
                        (process_size, process_size), resample=Image.Resampling.LANCZOS
                    ),
                    mask_image=mask_image.resize((process_size, process_size)),
                    strength=strength_slider.value,
                    num_inference_steps=step_input.value,
                    guidance_scale=guidance_input.value,
                )["sample"]
        else:
            with autocast("cuda"):
                images = text2img(
                    prompt=base.text_input.value,
                    height=process_size,
                    width=process_size,
                )["sample"]
        out = base.sel_buffer.copy()
        out[:, :, 0:3] = np.array(
            images[0].resize(
                (base.selection_size, base.selection_size),
                resample=Image.Resampling.LANCZOS,
            )
        )
        out[:, :, -1] = 255
        base.fill_selection(out)
        with hold_canvas():
            base.draw_selection_box()


def export_button_clicked(btn):
    with base.output:
        base.output.clear_output()
        img = base.export()
        pil = Image.fromarray(img)
        time_str = time.strftime("%Y%m%d_%H%M%S")
        pil.save(f"outpaint_{time_str}.png")
        print(f"Canvas saved to outpaint_{time_str}.png")
        display(pil)


base.run_button.on_click(run_outpaint)
base.export_button.on_click(export_button_clicked)


## have fun here

In [ ]:
display_lst=base.display()
display_lst.insert(-1,widgets.HBox([resize_check,step_input,guidance_input,strength_slider]))
for item in display_lst:
    display(item)